In [19]:
import requests
import xmltodict
import certifi

url = "https://boardgamegeek.com/xmlapi2/thing"
params = {
    "id": 1,
    "type": "boardgame",
    "stats": 1
}

# Option 1: Using proper certificate verification (recommended)
# Option 2: Disable SSL verification (less secure)
response = requests.get(
    url=url,
    params=params,
    verify=False
)

# Suppress only the InsecureRequestWarning if using Option 2
# import urllib3
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

response_data = xmltodict.parse(response.content)

c:\Users\hcvaldrez\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'boardgamegeek.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [20]:
response_data

{'items': {'@termsofuse': 'https://boardgamegeek.com/xmlapi/termsofuse',
  'item': {'@type': 'boardgame',
   '@id': '1',
   'thumbnail': 'https://cf.geekdo-images.com/rpwCZAjYLD940NWwP3SRoA__thumb/img/YT6svCVsWqLrDitcMEtyazVktbQ=/fit-in/200x150/filters:strip_icc()/pic4718279.jpg',
   'image': 'https://cf.geekdo-images.com/rpwCZAjYLD940NWwP3SRoA__original/img/yR0aoBVKNrAmmCuBeSzQnMflLYg=/0x0/filters:format(jpeg)/pic4718279.jpg',
   'name': [{'@type': 'primary', '@sortindex': '5', '@value': 'Die Macher'},
    {'@type': 'alternate', '@sortindex': '1', '@value': '德国大选'},
    {'@type': 'alternate', '@sortindex': '1', '@value': '디 마허'}],
   'description': 'Die Macher is a game about seven sequential political races in different regions of Germany. Players are in charge of national political parties, and must manage limited resources to help their party to victory. The winning party will have the most victory points after all the regional elections. There are four different ways of scoring vi